# Домашнее задание по Speech

Привет! Пора закрепить тему ASR на практике. В этой домашке тебя ждёт два крутых этапа:

1. Собери ASR с нуля:
  * Научишься обрабатывать аудио, проектировать нейросети для задачи ASR,
получать предсказания и считать метрики вроде WER.
  * Создашь модель, которая превращает речь в текст. Никакой магии — только код, данные и тренировки.

2. Сделай её «стриминговой»:
  * Переделаешь модель так, чтобы она работала в реальном времени (как в Zoom или Алисе).
  * Разберёшься, как уменьшить задержки и балансировать между скоростью и точностью.

**Чего получится:** готовый ASR, который понимает и записанную речь, и поток с микрофона. Потрогаешь torch, torchaudio, librosa и поймёшь, как устроены голосовые ассистенты.



---



# Offline ASR

### Подготовим различные утилиты и пакеты



In [29]:
!pip install levenshtein

In [30]:
import os
os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'

import torch
import string
import librosa
import torchaudio
import Levenshtein
import numpy as np
import torch.nn as nn
import torch.utils.data as data
import torch.nn.functional as F

from tqdm import tqdm
from torch import optim
from typing import Tuple, List, Dict, Union

In [31]:
torch.manual_seed(7)

if torch.backends.mps.is_available():
    print("MPS (Apple GPU) found! 🍏🚀")
    device = "mps"
else:
    print("Only CPU found! 💻")
    device = "cpu"

MPS (Apple GPU) found! 🍏🚀




---



### Метрики

Подробнее почитать про то, как работает WER / CER можно тут:
* https://huggingface.co/learn/audio-course/ru/chapter5/evaluation

In [32]:
def cer(reference: str, hypothesis: str, ignore_case: bool = False, remove_space: bool = False) -> float:
    """Вычисляет Character Error Rate (CER) между эталонной и проверяемой строками.

    CER - это метрика, которая сравнивает эталонный текст (оригинал) с проверяемым текстом
    (предсказание) на уровне символов. Вычисляет редакционное расстояние (расстояние Левенштейна)
    между строками и нормализует его по длине эталонного текста.

    Аргументы:
        reference (str): Эталонная строка для сравнения (оригинальный текст).
        hypothesis (str): Проверяемая строка (предсказанный текст).
        ignore_case (bool, опционально): Если True, игнорирует регистр при сравнении. По умолчанию False.
        remove_space (bool, опционально): Если True, удаляет пробелы перед сравнением. По умолчанию False.

    Возвращает:
        float: Значение Character Error Rate от 0 до 1, где 0 означает полное совпадение.

    Выбрасывает:
        ValueError: Если длина эталонной строки после предобработки равна 0.
    """
    edit_distance, ref_len = char_errors(
        reference, hypothesis, ignore_case, remove_space
    )

    if ref_len == 0:
        raise ValueError("Length of reference should be greater than 0.")

    cer = float(edit_distance) / ref_len
    return cer


def wer(reference: str, hypothesis: str, ignore_case: bool = False, remove_space: bool = False, delimiter: str = " ") -> float:

    """Вычисляет Word Error Rate (WER) между эталонным и проверяемым текстом.

    WER - это метрика, которая сравнивает эталонный текст с проверяемым текстом
    на уровне слов. Вычисляет редакционное расстояние (расстояние Левенштейна)
    между последовательностями слов и нормализует его по количеству слов в эталонном тексте.

    Аргументы:
        reference (str): Эталонный текст для сравнения (оригинальный текст).
        hypothesis (str): Проверяемый текст (предсказанный текст).
        ignore_case (bool, опционально): Если True, игнорирует регистр при сравнении.
                                        По умолчанию False.
        delimiter (str, опционально): Разделитель для токенизации текста на слова.
                                     По умолчанию пробел (" ").

    Возвращает:
        float: Значение Word Error Rate от 0 до 1, где:
               - 0 означает полное совпадение
               - 1 означает полное несовпадение

    Выбрасывает:
        ValueError: Если количество слов в эталонном тексте равно 0 после обработки.

    Пример:
        >>> wer("это тест", "этот тест")
        0.5  # одна ошибка (замена) при двух словах в эталоне
    """
    edit_distance, ref_len = word_errors(reference, hypothesis, ignore_case, delimiter)

    if ref_len == 0:
        raise ValueError("Reference's word number should be greater than 0.")

    wer = float(edit_distance) / ref_len
    return wer


def char_errors(reference: str, hypothesis: str, ignore_case: bool = False, remove_space: bool = False) -> Tuple[float, int]:
    """Вычисляет количество ошибок и длину эталонной строки после предобработки.

    Функция выполняет сравнение двух строк на уровне символов с учетом опциональных
    преобразований и возвращает:
    1) Расстояние Левенштейна (редакционное расстояние) между строками
    2) Длину эталонной строки после обработки

    Аргументы:
        reference (str): Эталонная строка (оригинальный текст)
        hypothesis (str): Проверяемая строка (предсказанный текст)
        ignore_case (bool, опционально): Флаг игнорирования регистра. Если True,
                                       сравнение происходит в нижнем регистре.
                                       По умолчанию False.
        remove_space (bool, опционально): Флаг удаления пробелов. Если True,
                                        все пробелы удаляются перед сравнением.
                                        По умолчанию False.

    Возвращает:
        tuple[float, int]: Кортеж из двух элементов:
        - float: Расстояние Левенштейна между строками
        - int: Длина эталонной строки после обработки

    Примеры:
        >>> char_errors("тест", "тест")
        (0.0, 4)  # полное совпадение

        >>> char_errors("Тест", "тест", ignore_case=True)
        (0.0, 4)  # совпадение при игнорировании регистра

        >>> char_errors("т е с т", "тест", remove_space=True)
        (0.0, 4)  # совпадение после удаления пробелов
    """
    if ignore_case:
        reference = reference.lower()
        hypothesis = hypothesis.lower()

    join_char = " "
    if remove_space:
        join_char = ""

    reference = join_char.join(filter(None, reference.split(" ")))
    hypothesis = join_char.join(filter(None, hypothesis.split(" ")))

    edit_distance = Levenshtein.distance(reference, hypothesis)
    return float(edit_distance), len(reference)


def word_errors(reference: str, hypothesis: str, ignore_case: bool = False, delimiter: str = " ") -> Tuple[float, int]:
    """Вычисляет количество ошибок и число слов в эталонном тексте после предобработки.

    Функция выполняет сравнение двух текстов на уровне слов с учетом опциональных
    преобразований и возвращает:
    1) Расстояние Левенштейна между последовательностями слов
    2) Количество слов в эталонном тексте после обработки

    Аргументы:
        reference (str): Эталонный текст (оригинальный текст)
        hypothesis (str): Проверяемый текст (предсказанный текст)
        ignore_case (bool, опционально): Флаг игнорирования регистра. Если True,
                                       сравнение происходит в нижнем регистре.
                                       По умолчанию False.
        delimiter (str, опционально): Разделитель для токенизации текста на слова.
                                    По умолчанию пробел (" ").

    Возвращает:
        tuple[float, int]: Кортеж из двух элементов:
        - float: Расстояние Левенштейна между последовательностями слов
        - int: Количество слов в эталонном тексте после обработки

    Примеры:
        >>> word_errors("это тест", "это тест")
        (0.0, 2)  # полное совпадение

        >>> word_errors("Это Тест", "это тест", ignore_case=True)
        (0.0, 2)  # совпадение при игнорировании регистра

        >>> word_errors("это,тест", "это тест", delimiter=",")
        (1.0, 2)  # различие при измененном разделителе

    Примечание:
        Для расчета расстояния между последовательностями слов используется
        алгоритм Левенштейна, применяемый к спискам слов.
    """
    if ignore_case:
        reference = reference.lower()
        hypothesis = hypothesis.lower()

    ref_words = reference.split(delimiter)
    hyp_words = hypothesis.split(delimiter)

    edit_distance = Levenshtein.distance(ref_words, hyp_words)
    return float(edit_distance), len(ref_words)


def avg_wer(wer_scores: List[float], combined_ref_len: Union[int, float]) -> float:
    return float(sum(wer_scores)) / float(combined_ref_len)

Всякое остальное...

In [33]:
def save_checkpoint(model, checkpoint_name, path="./"):
    torch.save(
        {"model_state_dict": model.state_dict()}, os.path.join(path, checkpoint_name)
    )


def load_checkpoint(model, path, checkpoint_name, device):
    checkpoint = torch.load(os.path.join(path, checkpoint_name), map_location=device)
    model.load_state_dict(checkpoint["model_state_dict"])



---



### Напишем свой CTC ASR!

Итак, суть задания - написать и обучить свой ASR с нуля на датасете [librispeech](https://www.openslr.org/12) clean

*librispeech clean* - это датасет с прочтением аудиокниг в достаточно чистых условиях.

На этом этапе домашки вам необходимо будет сделать следующие действия:
1. Написать Greedy Decoder - **0.5 балла**
2. Написать и обучить модель ASR - **4 балла**

Ожидаю, что в домашке вы сделаете ASR при помощи CTC лосса, но в целом другие схемы не запрещаются.

[Sample Rate](https://en.wikipedia.org/wiki/Sampling_(signal_processing)#Sampling_rate) датасета - 16kHz - эта информация будет вам полезна для подготовки датасета.

In [34]:
if not os.path.isdir("./data"):
    os.makedirs("./data")

train_dataset = torchaudio.datasets.LIBRISPEECH(
    "./data", url="train-clean-100", download=False
)
test_dataset = torchaudio.datasets.LIBRISPEECH(
    "./data", url="test-clean", download=False
)

SAMPLE_RATE = 16_000

Подготовим заранее простой токенизатор, который состоит из букв алфавита и бланк символа.

Подробнее про декодинг CTC декодинг [тут](https://distill.pub/2017/ctc/)

In [35]:
BLANK_SYMBOL = "_"

class Tokenizer:
    """Класс для преобразования между символами и их числовыми представлениями.

    Обеспечивает взаимно-однозначное соответствие между символами алфавита
    и их числовыми индексами. Поддерживает основные буквы английского алфавита,
    апостроф, пробел и специальный blank-символ для CTC-алгоритма.

    Атрибуты:
        char_map (Dict[str, int]): Словарь символ -> индекс
        index_map (Dict[int, str]): Словарь индекс -> символ
    """

    def __init__(self) -> None:
        """Инициализирует словари преобразования символов.

        Создает соответствия для:
        - Апострофа ('), пробела ( )
        - Строчных букв английского алфавита (a-z)
        - Специального blank-символа (по умолчанию '_')
        """
        self.char_map: Dict[str, int] = {}
        self.index_map: Dict[int, str] = {}
        for i, ch in enumerate(
            ["'", " "] + list(string.ascii_lowercase) + [BLANK_SYMBOL]
        ):
            self.char_map[ch] = i
            self.index_map[i] = ch

    def text_to_indices(self, text: str) -> List[int]:
        """Преобразует строку текста в список числовых индексов.

        Args:
            text: Входная строка для преобразования

        Returns:
            Список индексов, соответствующих символам входной строки

        Raises:
            KeyError: если встретится символ, отсутствующий в char_map
        """
        return [self.char_map[ch] for ch in text]

    def indices_to_text(self, labels: List[int]) -> str:
        """Преобразует список индексов обратно в строку.

        Args:
            labels: Список числовых индексов

        Returns:
            Строка, составленная из символов, соответствующих индексам

        Raises:
            KeyError: если встретится индекс, отсутствующий в index_map
        """
        return "".join([self.index_map[i] for i in labels])

    def get_symbol_index(self, sym: str) -> int:
        """Возвращает индекс для указанного символа.

        Args:
            sym: Символ, для которого требуется получить индекс

        Returns:
            Числовой индекс символа

        Raises:
            KeyError: если символ отсутствует в char_map
        """
        return self.char_map[sym]



---



### Greedy Decoding - 0.5 балла

Вам необходимо написать свой простенький Greedy Decoder для ASR. Правила можно посмотреть на лекции / загуглить. Для проверки будут использоваться тесты.

In [36]:
def greedy_decoder(
    output: torch.Tensor,
    input_lengths: List[int],
    labels: List[torch.Tensor],
    label_lengths: List[int],
    tokenizer: Tokenizer,
    blank_id: int = BLANK_SYMBOL
) -> Tuple[np.ndarray, np.ndarray]:
    """Реализует greedy-декодирование выхода CTC-модели с обработкой пакета данных.

    Функция выполняет:
    1. Декодирование предсказаний модели (удаление повторяющихся символов и blank-токенов)
    2. Декодирование целевых последовательностей (targets)
    3. Возвращает текстовые представления предсказаний и целей

    Аргументы:
        output (torch.Tensor): Тензор выхода модели размером [B, T, C],
                              где B - размер пакета, T - временные шаги, C - классы
        input_lengths (List[int]): Длины последовательностей для каждого элемента пакета
        labels (List[torch.Tensor]): Целевые последовательности (индексы символов)
        label_lengths (List[int]): Длины целевых последовательностей
        tokenizer (Tokenizer): Объект для преобразования индексов в текст
        blank_id (int, optional): Индекс blank-символа. По умолчанию BLANK_SYMBOL

    Возвращает:
        Tuple[np.ndarray, np.ndarray]: Кортеж из двух массивов:
            - decoded_outputs: Декодированные предсказания модели (текст)
            - decoded_targets: Декодированные целевые последовательности (текст)

    Пример:
        >>> output = torch.randn(2, 50, 30)  # пакет из 2 примеров
        >>> input_lengths = [45, 50]
        >>> labels = [torch.tensor([1,2,3]), torch.tensor([4,5,6])]
        >>> label_lengths = [3, 3]
        >>> preds, targets = greedy_decoder(output, input_lengths, labels, label_lengths, tokenizer)
        >>> print(preds[0])  # декодированное первое предсказание
    """
    decoded_outputs = []
    decoded_targets = []

    max_probs = torch.argmax(output, dim=-1)

    for i in range(output.size(0)):
        raw_prediction = max_probs[i][:input_lengths[i]].tolist()

        decoded = []
        previous = None
        for p in raw_prediction:
            if p != previous and p != tokenizer.get_symbol_index(blank_id):
                decoded.append(p)
            previous = p

        decoded_outputs.append(tokenizer.indices_to_text(decoded))

        target = labels[i][:label_lengths[i]].tolist()
        decoded_targets.append(tokenizer.indices_to_text(target))

    return np.array(decoded_outputs), np.array(decoded_targets)


Тесты

In [37]:
import pytest
import torch
from typing import List, Tuple
from unittest.mock import Mock

# Мок для tokenizer
class MockTokenizer(Tokenizer):
    def __init__(self):
        self.vocab = {'a': 0, 'b': 1, '_': 2}
        self.reverse_vocab = {v: k for k, v in self.vocab.items()}

    def get_symbol_index(self, symbol: str) -> int:
        return self.vocab[symbol]

    def indices_to_text(self, indices: List[int]) -> str:
        return ''.join([self.reverse_vocab[i] for i in indices])

mock_tokenizer = MockTokenizer()

test_cases = [
    # Базовый случай: нет повторов и бланков
    {
        "output": torch.tensor([[[0.9, 0.1, 0.0], [0.1, 0.8, 0.1], [0.2, 0.7, 0.1]]]),
        "input_lengths": [3],
        "labels": [torch.tensor([0, 1, 1])],
        "label_lengths": [3],
        "expected_output": ["ab"],
        "expected_target": ["abb"]
    },
    # Повторы и бланки
    {
        "output": torch.tensor([[[0.8, 0.1, 0.1], [0.1, 0.1, 0.8], [0.7, 0.2, 0.1], [0.1, 0.8, 0.1]]]),
        "input_lengths": [4],
        "labels": [torch.tensor([0, 1])],
        "label_lengths": [2],
        "expected_output": ["aab"],  # a, a, blank, b → aab (повтор a схлопывается)
        "expected_target": ["ab"]
    },
    # Все бланки
    {
        "output": torch.tensor([[[0.1, 0.1, 0.8], [0.2, 0.1, 0.7], [0.1, 0.2, 0.7]]]),
        "input_lengths": [3],
        "labels": [torch.tensor([])],
        "label_lengths": [3],
        "expected_output": [""],  # все токены - бланки
        "expected_target": [""]
    },
    # Обрезание по input_lengths
    {
        "output": torch.tensor([[[0.9, 0.0, 0.1], [0.8, 0.1, 0.1], [0.1, 0.9, 0.0], [0.1, 0.1, 0.8]]]),
        "input_lengths": [2],  # берем первые 2 элемента
        "labels": [torch.tensor([0, 1])],
        "label_lengths": [2],
        "expected_output": ["a"],  # a (первый), a (второй) → a (схлопывается)
        "expected_target": ["ab"]
    },
    # Пустые последовательности
    {
        "output": torch.tensor([[[0.1, 0.1, 0.8]]]),
        "input_lengths": [1],
        "labels": [torch.tensor([])],  # длина 0
        "label_lengths": [0],
        "expected_output": [""],
        "expected_target": [""]
    }
]

def test_greedy_decoder(test_case):
    decoded_outputs, decoded_targets = greedy_decoder(
        output=test_case["output"],
        input_lengths=test_case["input_lengths"],
        labels=test_case["labels"],
        label_lengths=test_case["label_lengths"],
        tokenizer=mock_tokenizer,
    )

    return decoded_outputs == test_case["expected_output"] and decoded_targets == test_case["expected_target"]

for i, test_case in enumerate(test_cases):
  result = test_greedy_decoder(test_case)
  print(f"Test {i}: {('Passed' if result else 'Failed')}!")

Test 0: Passed!
Test 1: Passed!
Test 2: Passed!
Test 3: Passed!
Test 4: Passed!




---



### Обучим свой ASR - 4 балла

В этой части домашки вам будет необходимо обучить свой ASR на датасете librispeech clean.

Вам необходимо будет сделать следующее:
1. Написать препроцессинг аудио для трейна и для теста. Нужно правильно получить спектрограмму, в трейн можете добавить аугментаций и тд.
2. Придумать архитектуру для вашей модели. Можете вдохновляться тем, что мы обсуждали на лекции: [QuartzNet](https://arxiv.org/abs/1910.10261), [JasperNet](https://arxiv.org/abs/1904.03288), [Conformer](https://arxiv.org/abs/2005.08100). А также теми, которые не успели обсудить: [DeepSpeech 2](https://arxiv.org/abs/1512.02595), [LAS](https://arxiv.org/abs/1508.01211), и всем остальным, что сможете найти. Здесь вы не чем не ограничены, только вашими вычислительными ресурсами :)
3. Написать трейн / тест луп. Я бы предложил обратить внимание на следующие вещи:
  * важно, чтобы правильно считался [CTCLoss](https://pytorch.org/docs/stable/generated/torch.nn.CTCLoss.html) (опять же можете использовать любой другой), часто у людей там бывают проблемы с масками и длинами :)
  * не забывайте про возможность использовать [автокаст](https://pytorch.org/docs/stable/amp.html), это позволит вам выбить скор получше в условиях, ограниченных по ресурсам.
4. Обучить модель.

*Для зачета по заданию достаточно получить 60 WER.*

**Важно!** Следующее задание - переписать ваш ASR на стриминговую версию, поэтому проектируйте вашу архитектуру так, чтобы ее было возможно переписать на стриминг, а также чтобы это было удобно. Например, не стоит использовать bidirectional lstm, которая требует контекста из будущего.

In [38]:
mel_spec_transform = torchaudio.transforms.MelSpectrogram(
    sample_rate=16000,
    n_fft=400,
    win_length=400,
    hop_length=160,
    n_mels=80,
)

db_transform = torchaudio.transforms.AmplitudeToDB(stype="power", top_db=80)


def preprocess_audio_train(audiodata: torch.Tensor) -> torch.Tensor:
    mel_spec = mel_spec_transform(audiodata)
    log_mel_spec = db_transform(mel_spec)
    log_mel_spec = torchaudio.transforms.FrequencyMasking(freq_mask_param=15)(log_mel_spec)
    log_mel_spec = torchaudio.transforms.TimeMasking(time_mask_param=35)(log_mel_spec)

    return log_mel_spec.transpose(0, 1)

def preprocess_audio_test(audiodata: torch.Tensor) -> torch.Tensor:
    mel_spec = mel_spec_transform(audiodata)
    log_mel_spec = db_transform(mel_spec)

    return log_mel_spec.transpose(0, 1)

In [39]:
class Collate:
    """Класс для создания батчей из данных LibriSpeech.

    Обрабатывает аудиоданные и текст, формируя батчи для обучения/тестирования модели:
    - Преобразует аудио в спектрограммы (с аугментациями для обучения)
    - Токенизирует текст
    - Выравнивает последовательности в батче с помощью паддинга

    Атрибуты:
        data_type (str): Тип данных ("train" или "test"), определяет применение аугментаций
    """

    def __init__(self, data_type: str = "test") -> None:
        """Инициализирует коллатор.

        Args:
            data_type: Тип данных ("train" для обучения с аугментациями,
                     "test" для валидации/теста без аугментаций)
        """
        super(Collate, self).__init__()
        self.data_type = data_type

    def __call__(
        self, data: torchaudio.datasets.librispeech.LIBRISPEECH
    ) -> Tuple[torch.Tensor, torch.Tensor, List[int], List[int]]:
        """Обрабатывает список примеров и формирует батч.

        Args:
            data: Список кортежей (waveform, sample_rate, utterance, ...) из датасета LibriSpeech,
                 где waveform - аудиосигнал [1, samples], utterance - текстовая транскрипция

        Returns:
            Кортеж из:
            - spectrograms: Тензор спектрограмм [batch_size, 1, max_time, n_mels]
            - labels: Тензор с токенизированными текстами [batch_size, max_label_len]
            - input_lengths: Список длин спектрограмм для каждого примера
            - label_lengths: Список длин текстовых последовательностей

        Raises:
            Exception: Если data_type не "train" или "test"

        Пример:
            >>> collate_fn = Collate(data_type="train")
            >>> batch = collate_fn(dataset_samples)
            >>> specs, labels, spec_lens, label_lens = batch
        """
        spectrograms = []
        labels = []
        input_lengths = []
        label_lengths = []

        for waveform, _, utterance, _, _, _ in data:
            # Обработка аудио
            if self.data_type == "train":
                spec = preprocess_audio_train(waveform)  # [TIME, CHANNELS]
            elif self.data_type == "test":
                spec = preprocess_audio_test(waveform)  # [TIME, CHANNELS]

            else:
                raise Exception("data_type should be train or test")

            if spec.dim() == 3:
    # Проверяем, есть ли лишенный размер в середине
                if spec.shape[1] == 1:
                  spec = spec.squeeze(1)  # Уберем размерность 1: [80, time]
                elif spec.shape[0] == 1:
                  spec = spec.squeeze(0)  # Другой вариант
    # Теперь spec 2D: [80, time]
                spec = spec.transpose(0, 1)  # [time, 80]
            elif spec.dim() == 2:
                spec = spec.transpose(0, 1)  # [time, n_mels]

            spectrograms.append(spec)

            label = torch.Tensor(tokenizer.text_to_indices(utterance.lower()))

            labels.append(label)
            input_lengths.append(spec.shape[0])
            label_lengths.append(len(label))

        # Упаковка в батч с паддингом
        spectrograms = nn.utils.rnn.pad_sequence(
            spectrograms, batch_first=True
        ) # [BATCH_SIZE, 1, TIME, CHANNELS]
        spectrograms = spectrograms.unsqueeze(1)
        labels = nn.utils.rnn.pad_sequence(labels, batch_first=True)

        return spectrograms, labels, input_lengths, label_lengths



In [40]:
class ASRModel(nn.Module):
    def __init__(self, n_mels: int, n_tokens: int):
        super().__init__()

        self.encoder = nn.Sequential(
            nn.Conv1d(n_mels, 128, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm1d(128),
            nn.ReLU(),

            nn.Conv1d(128, 128, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm1d(128),
            nn.ReLU(),

            nn.Conv1d(128, 256, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm1d(256),
            nn.ReLU(),
        )

        self.classifier = nn.Sequential(
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Linear(256, n_tokens)
        )

    def forward(self, x: torch.Tensor, xlen: List[int]) -> Tuple[torch.Tensor, List[int]]:
        x = x.squeeze(1).transpose(1, 2)
        x = self.encoder(x)
        x = x.transpose(1, 2)
        logits = self.classifier(x)

        return logits, xlen


In [41]:
def train(
    model: nn.Module,
    device: str,
    train_loader: data.DataLoader,
    criterion: nn.Module,
    optimizer: torch.optim.Optimizer,
    scheduler,
    epoch: int,
) -> None:
    model.train()
    data_len = len(train_loader.dataset)

    for batch_idx, _data in tqdm(enumerate(train_loader), total=len(train_loader)):
        spectrograms, labels, input_lengths, label_lengths = _data
        spectrograms, labels = spectrograms.to(device), labels.to(device)
        outputs, output_lengths = model(spectrograms, input_lengths)
        outputs = outputs.transpose(0, 1)
        loss = criterion(outputs, labels, output_lengths, label_lengths)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if scheduler is not None:
            scheduler.step()


        if batch_idx % 10 == 0 or batch_idx == data_len:
            print(
                "Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                    epoch,
                    batch_idx * len(spectrograms),
                    data_len,
                    100.0 * batch_idx / len(train_loader),
                    loss.item(),
                )
            )


def test(
    model: nn.Module,
    device: str,
    test_loader: data.DataLoader,
    criterion: nn.Module,
    tokenizer: Tokenizer,
    epoch: int,
) -> None:
    print("Beginning eval...")
    model.eval()
    test_loss = 0
    test_cer, test_wer = [], []
    with torch.no_grad():
        for i, _data in tqdm(enumerate(test_loader), total=len(test_loader)):
            spectrograms, labels, input_lengths, label_lengths = _data
            spectrograms, labels = spectrograms.to(device), labels.to(device)

            outputs, output_lengths = model(spectrograms, input_lengths)
            outputs = outputs.transpose(0, 1)

            loss = criterion(outputs, labels, output_lengths, label_lengths)
            matrix = outputs.softmax(dim=-1)

            test_loss += loss.item() / len(test_loader)

            decoded_preds, decoded_targets = greedy_decoder(
                matrix.transpose(0, 1), input_lengths, labels, label_lengths, tokenizer
            )

            for j in range(len(decoded_preds)):
                test_cer.append(cer(decoded_targets[j], decoded_preds[j]))
                test_wer.append(wer(decoded_targets[j], decoded_preds[j]))

    avg_cer = sum(test_cer) / len(test_cer)
    avg_wer = sum(test_wer) / len(test_wer)
    print(
        "Epoch: {:d}, Test set: Average loss: {:.4f}, Average CER: {:4f} Average WER: {:.4f}\n".format(
            epoch, test_loss, avg_cer, avg_wer
        )
    )

In [16]:
BATCH_SIZE = 16
EPOCHS = 10

tokenizer = Tokenizer()

train_collate_fn = Collate(data_type="train")
test_collate_fn = Collate(data_type="test")

# Define Dataloyour training and test data loaders
kwargs = {"num_workers": 2, "pin_memory": True} if device == "cuda" else {}
train_loader = data.DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    collate_fn=train_collate_fn,
    **kwargs,
)

kwargs = {"num_workers": 1, "pin_memory": True} if device == "cuda" else {}
test_loader = data.DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    collate_fn=test_collate_fn,
    **kwargs,
)

n_mels = 80
n_tokens = len(tokenizer.char_map)
blank_id = tokenizer.char_map[BLANK_SYMBOL]
criterion = nn.CTCLoss(blank=blank_id, zero_infinity=True)
model = ASRModel(n_mels=n_mels, n_tokens=n_tokens).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.9)


for epoch in range(1, EPOCHS + 1):
    train(model, device, train_loader, criterion, optimizer, scheduler, epoch)
    save_checkpoint(model, checkpoint_name="last_checkpoint.tar")
    test(model, device, test_loader, criterion, tokenizer, epoch)

save_checkpoint(model, checkpoint_name="last_checkpoint.tar")

  0%|                                          | 1/1784 [00:00<11:06,  2.67it/s]

Train Epoch: 1 [0/28539 (0%)]	Loss: -5.080374


  1%|▎                                        | 12/1784 [00:02<05:06,  5.77it/s]

Train Epoch: 1 [160/28539 (1%)]	Loss: 6.771354


  1%|▌                                        | 22/1784 [00:04<05:12,  5.63it/s]

...

Train Epoch: 10 [28480/28539 (100%)]	Loss: 4.388375


100%|███████████████████████████████████████| 1784/1784 [05:39<00:00,  5.25it/s]


Beginning eval...


100%|█████████████████████████████████████████| 164/164 [00:13<00:00, 11.77it/s]

Epoch: 10, Test set: Average loss: 5.0786, Average CER: 0.929737 Average WER: 1.0058





---



# Streaming

Далее ваша задание заключается в том, чтобы переписать ваш выше написанный ASR, который выбил <60 WER на стриминговую версию.

> Не пройдет схема типа написать STFT + линейный слой, выбить выше 100+ WER, а потом просто получить за это задание 4 балла. Это задание **засчитывается только если ваша архитектура выше реально рабочая** и может решать хотя бы как-то задачу ASR. Предварительное условие, что задание по стримингу засчитывается, если вы набрали <60 WER, но гвоздями не прибито, можем поторговаться, например, если вы написали рабочую архитектуру, переписали на стриминг версию, но из-за каких-то проблем с видеокартами / ресурсами и тд, не получилось обучить модель.

Оценка заданий:
1. Переписать на стриминг вариант конволюцию - **0.5 балла**
2. Переписать Attention на стриминговую версию - **1 балл**
3. Переписать Attention с chunked mask на стриминговую версию - **1 балл**
4. Переписать ASR на стриминговую версию - **4 балла**



---



### Разберемся как работает стриминг!

Для этого сделаем несколько упражнений по переписыванию операций, чтобы вам было проще переписывать вашу модель.

##### Пример STFT

Чтобы показать вам наглядный пример, на основе которого можно делать другие переписывания, то покажу пример того, как можно переписать STFT в стриминг режим.

In [42]:
import torch

# Устанавливаем seed для воспроизводимости результатов
torch.manual_seed(1231)

# Создаем случайный аудиосигнал длиной 16000 сэмплов (1 секунда при 16кГц)
sample_audio = torch.rand(16000) # Пример случайного аудио

# Параметры STFT преобразования
n_fft = 512       # Размер окна преобразования
hop_length = n_fft // 2  # Шаг между окнами (50% перекрытие)

# Создаем кастомное оконная функция (модифицированное синусоидальное окно)
window = torch.sin(
    0.5 * torch.pi * (torch.arange(n_fft) + 0.5) / hop_length
)
window = torch.sin(0.5 * torch.pi * window ** 2)  # Дополнительное преобразование окна

# 1. Оффлайн STFT обработка (обычный подход)
offline_stft_out = torch.stft(
    sample_audio,
    n_fft=512,
    hop_length=hop_length,
    win_length=n_fft,
    window=window,
    center=False,    # Не добавляем padding по краям
    onesided=True,   # Используем только положительные частоты
    normalized=False,# Без нормализации
    return_complex=True  # Возвращаем комплексные числа
)

# 2. Потоковая обработка (имитация потоковой обработки-а)

# Разбиваем аудио на фреймы с шагом hop_length
frames = sample_audio.split(hop_length)

# Здесь будем накапливать результаты обработки каждого фрейма
output_frames = []

# Буфер для хранения предыдущего фрейма (нужен для перекрытия окон)
previous_frame = None

# Имитация потоковой обработки
for frame in frames:
    # Обрабатываем только если есть предыдущий фрейм и текущий полного размера
    if previous_frame is not None and frame.shape[0] == hop_length:
        # Соединяем предыдущий и текущий фрейм (перекрытие окон)
        windowed_frame = torch.cat([previous_frame, frame]) * window

        # Вычисляем FFT только для положительных частот
        rfft_frame = torch.fft.rfft(windowed_frame)

        # Сохраняем результат обработки фрейма
        output_frames.append(rfft_frame)

    # Сохраняем текущий фрейм для следующей итерации
    previous_frame = frame

# Собираем все обработанные фреймы в один тензор
# Транспонируем, чтобы привести к тому же формату, что и в offline_stft_out
streaming_stft_out = torch.stack(output_frames).T

# Сравниваем результаты оффлайн и потоковой обработки
print("Результаты совпадают:", torch.allclose(offline_stft_out, streaming_stft_out))

Результаты совпадают: True




---



##### Перепишем конволюцию в стриминг вариант - **0.5 балла**


В этом задании вам необходимо реализовать стриминговую реализацию конволюции по аналогии с примером выше.

In [61]:
# Параметры модели
N_FEATS = 128  # Количество входных каналов

# Пример случайного входного сигнала (формат [batch, channels, time])
offline_stft_out = torch.rand(1, N_FEATS, 512)

# Оффлайн-свертка (обычная обработка всего сигнала)
conv = nn.Conv1d(N_FEATS, 1024, 3, stride=1)
conv_out_offline = conv(F.pad(offline_stft_out, (1, 1)))  # Добавляем паддинг для сохранения размера

# Разбиваем входной сигнал на отдельные кадры для потоковой обработки
frames = torch.split(offline_stft_out, 1, dim=2)  # Размер каждого кадра: [1, N_FEATS, 1]

# Сюда будем сохранять результаты потоковой обработки
output_frames = []

###################### YOUR CODE ############################
buffer = [torch.zeros(BATCH_SIZE, N_FEATS, 1) for _ in range(PADDING)]

###################### YOUR CODE ############################

# 2. Реализуйте логику потоковой обработки:
for frame in frames:
    buffer.append(frame)
    if len(buffer) < KERNEL_SIZE:
        continue
    input_chunk = torch.cat(buffer[-KERNEL_SIZE:], dim=2)
    out = conv(input_chunk)
    output_frames.append(out.squeeze(-1))

buffer.append(torch.zeros(BATCH_SIZE, N_FEATS, 1))
input_chunk = torch.cat(buffer[-KERNEL_SIZE:], dim=2)
out = conv(input_chunk)
output_frames.append(out.squeeze(-1))

# Сравниваем результаты
conv_stacked = torch.stack(output_frames, dim=-1)  # Собираем все выходные кадры
print("Результаты совпадают:",
      torch.allclose(conv_out_offline[:, :, :conv_stacked.shape[-1]], conv_stacked, atol=1e-3))

Результаты совпадают: True




---



##### Перепишем attention в стриминг вариант - **1 балл**


Накидал за вас простейшую имплементацию MHA

In [63]:
import math
import torch

from torch import nn


class ScaleDotProductAttention(nn.Module):
    """
    compute scale dot product attention

    Query : given sentence that we focused on (decoder)
    Key : every sentence to check relationship with Qeury(encoder)
    Value : every sentence same with Key (encoder)
    """

    def __init__(self):
        super(ScaleDotProductAttention, self).__init__()
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, q, k, v, mask=None, e=1e-12):
        # input is 4 dimension tensor
        # [batch_size, head, length, d_tensor]
        batch_size, head, length, d_tensor = k.size()

        # 1. dot product Query with Key^T to compute similarity
        k_t = k.transpose(2, 3)  # transpose
        score = (q @ k_t) / math.sqrt(d_tensor)  # scaled dot product

        # 2. apply masking (opt)
        if mask is not None:
            score = score.masked_fill(mask == 0, -10000)

        # 3. pass them softmax to make [0, 1] range
        score = self.softmax(score)

        # 4. multiply with Value
        v = score @ v

        return v, score


class MultiHeadAttention(nn.Module):

    def __init__(self, d_model=512, n_head=8):
        super(MultiHeadAttention, self).__init__()
        self.n_head = n_head
        self.attention = ScaleDotProductAttention()
        self.w_q = nn.Linear(d_model, d_model)
        self.w_k = nn.Linear(d_model, d_model)
        self.w_v = nn.Linear(d_model, d_model)
        self.w_concat = nn.Linear(d_model, d_model)

    def forward(self, q, k, v, mask=None):
        # 1. dot product with weight matrices
        q, k, v = self.w_q(q), self.w_k(k), self.w_v(v)

        # 2. split tensor by number of heads
        q, k, v = self.split(q), self.split(k), self.split(v)

        # 3. do scale dot product to compute similarity
        out, attention = self.attention(q, k, v, mask=mask)

        # 4. concat and pass to linear layer
        out = self.concat(out)
        out = self.w_concat(out)

        return out

    def split(self, tensor):
        """
        split tensor by number of head

        :param tensor: [batch_size, length, d_model]
        :return: [batch_size, head, length, d_tensor]
        """
        batch_size, length, d_model = tensor.size()

        d_tensor = d_model // self.n_head
        tensor = tensor.view(batch_size, length, self.n_head, d_tensor).transpose(1, 2)
        # it is similar with group convolution (split by number of heads)

        return tensor

    def concat(self, tensor):
        """
        inverse function of self.split(tensor : torch.Tensor)

        :param tensor: [batch_size, head, length, d_tensor]
        :return: [batch_size, length, d_model]
        """
        batch_size, head, length, d_tensor = tensor.size()
        d_model = head * d_tensor

        tensor = tensor.transpose(1, 2).contiguous().view(batch_size, length, d_model)
        return tensor

В этом задании вам необходимо реализовать стриминговую имплементацию для Multi Head Attention с tril маской.

In [65]:
import torch
torch.manual_seed(1111)

# Параметры модели
d_model = 512    # Размерность модели
n_heads = 8      # Количество голов внимания
batch_size = 1    # Размер батча
seq_len = 100     # Длина последовательности
n_blocks = 3      # Количество блоков внимания

# Создаем случайный входной тензор [batch, seq_len, d_model]
input_full = torch.randn(batch_size, seq_len, d_model)

# Инициализируем несколько блоков внимания
mha_list = [MultiHeadAttention(d_model, n_heads) for _ in range(n_blocks)]

def run_mha(x, mask=None):
    """Применяет все блоки внимания последовательно"""
    for block in mha_list:
        x = block(x, x, x, mask)
    return x

# Оффлайн обработка (весь входной тензор сразу)
mask = torch.tril(torch.ones(seq_len, seq_len))  # Нижнетреугольная маска
offline_mask = mask.view(1, 1, seq_len, seq_len).repeat(batch_size, n_heads, 1, 1)
offline_out = run_mha(input_full, offline_mask)

# Потоковая обработка - разбиваем на отдельные токены
input_chunks = torch.split(input_full, 1, dim=1)  # Разделяем по временной оси

###################### YOUR CODE ############################
cached_k = [ [] for _ in range(n_blocks) ]
cached_v = [ [] for _ in range(n_blocks) ]

###################### YOUR CODE ############################

streaming_out = []  # Здесь будем сохранять результаты

for t, chunk in enumerate(input_chunks):
    x = chunk

    for i, block in enumerate(mha_list):
        q = block.w_q(x) 
        k = block.w_k(x)
        v = block.w_v(x)
        q = block.split(q)
        k = block.split(k)
        v = block.split(v)

        if t == 0:
            cached_k[i] = [k]
            cached_v[i] = [v]
        else:
            cached_k[i].append(k)
            cached_v[i].append(v)

        full_k = torch.cat(cached_k[i], dim=2)
        full_v = torch.cat(cached_v[i], dim=2)
        causal_mask = torch.ones((1, 1, 1, full_k.size(2)), dtype=torch.bool)
        out, _ = block.attention(q, full_k, full_v, mask=causal_mask)
        x = block.concat(out)
        x = block.w_concat(x)

    streaming_out.append(x.squeeze(1)) 

# Собираем все предсказания в один тензор
streaming_out = torch.stack(streaming_out, dim=1)

# Проверяем совпадение с оффлайн версией
print("Результаты совпадают:", torch.allclose(offline_out, streaming_out, atol=1e-3))

Результаты совпадают: True




---



##### Перепишем attention в стриминг вариант с chunked mask-ой - **1 балл**

Теперь, когда вы разобрались, как сделать стриминговый атеншен с tril маской, то теперь реализуем стриминговый атеншен с блок-диагональной маской.

In [75]:
# Конфигурация модели
d_model = 512       # Размерность эмбеддингов
n_heads = 8         # Количество голов внимания
batch_size = 1       # Размер батча
seq_len = 100        # Длина последовательности
n_blocks = 5         # Количество слоев внимания

# Инициализация блоков внимания
mha_list = [MultiHeadAttention(d_model, n_heads) for _ in range(n_blocks)]

# Создаем случайный входной тензор [batch, seq_len, d_model]
input_full = torch.randn(batch_size, seq_len, d_model)

def run_mha(x, mask=None):
    """Применяет все блоки внимания последовательно"""
    for block in mha_list:
        x = block(x, x, x, mask)
    return x

# Настройки блочной диагональной маски
n_diag_blocks = 4            # Количество блоков в маске
block_size = seq_len // n_diag_blocks  # Размер каждого блока

# Оффлайн обработка с блочной диагональной маской
# Создаем маску из n_diag_blocks единичных матриц block_size x block_size
mask = torch.block_diag(
    *[torch.ones(block_size, block_size) for _ in range(n_diag_blocks)],
)
# Преобразуем маску в формат [batch, heads, seq_len, seq_len]
offline_mask = mask.view(1, 1, seq_len, seq_len).repeat(batch_size, n_heads, 1, 1)
# Применяем механизм внимания ко всей последовательности
offline_out = run_mha(input_full, mask)

# Потоковая обработка - разбиваем на отдельные токены
input_chunks = torch.split(input_full, 1, dim=1)  # Разделяем по временной оси

###################### YOUR CODE ############################
token_buffer = []
###################### YOUR CODE ############################

streaming_out = []  # Здесь сохраняем результаты обработки блоков

for t, chunk in enumerate(input_chunks):
    token_buffer.append(chunk) 
    current_block = torch.cat(token_buffer, dim=1)
    curr_len = current_block.shape[1]
    mask = torch.tril(torch.ones(curr_len, curr_len)).unsqueeze(0).unsqueeze(0)
    mask = mask.repeat(batch_size, n_heads, 1, 1)
    out = run_mha(current_block, mask)
    streaming_out.append(out[:, -1:, :])

    if curr_len == block_size:
        token_buffer = []
        
# Собираем все обработанные блоки в один тензор
streaming_out_stacked = torch.cat(streaming_out, dim=1)

# Проверяем совпадение с оффлайн версией
print("Результаты совпадают:",
      torch.allclose(offline_out[:, :streaming_out_stacked.shape[1]],
                   streaming_out_stacked, atol=1e-3))

Результаты совпадают: False




---



### Приступим к написанию стриминговой модели

In [95]:
class StreamingASRModel(nn.Module):
    def __init__(self, offline_model, window_size=1024, hop_length=256, preprocess_fn=None):
        super().__init__()
        self.offline_model = offline_model
        self.window_size = window_size
        self.hop_length = hop_length
        self.preprocess_fn = preprocess_fn

        self.sample_buffer = torch.zeros(0)

    def forward(self, frame: torch.Tensor):
        """
        Args:
            frame: Tensor формы [hop_length] — очередной фрейм аудио

        Returns:
            output (torch.Tensor or None): результат модели, если окно собрано, иначе None
        """
        self.sample_buffer = torch.cat([self.sample_buffer, frame], dim=0)

        if self.sample_buffer.shape[0] < self.window_size:
            return None

        audio_window = self.sample_buffer[:self.window_size] 
        features = self.preprocess_fn(audio_window)
        input_tensor = features.unsqueeze(0) 

        with torch.no_grad():
            output = self.offline_model(input_tensor) 
        self.sample_buffer = self.sample_buffer[self.hop_length:]

        return output

In [96]:
sample_audio = torch.randn(int(16000 * 5.5))  # Example random audio

In [97]:
model = model.cpu().eval()

# Offline example
preprocessing_out = preprocess_audio_test(sample_audio).unsqueeze(0)

offline_output, _ = model(preprocessing_out, [preprocessing_out.shape[2]])

Схема того, как проверяется стриминговая модель:
1. Берем предсказания модели в офлайн режиме
2. Берем предсказания модели в стриминг режиме. Нюанс! Вполне возможно, что вашей модели может быть необходим контекст из будущего, поэтому для проверки достаточно сравнения того куска тензора, который получил необходимый контекст.
3. Сравниваем, предсказания должны совпадать по `atol=1e-3`

In [ ]:
# Streaming example
# Впишите необходимую вам длину шага
HOP_LENGTH = 160

streaming_model = StreamingASRModel(model).eval()

frames = sample_audio.split(HOP_LENGTH)

# Every frame going to output
output_frames = []

# Streaming imitation
for frame in frames:
    output = streaming_model(frame)

    if output is not None:
        output_frames.append(output)

# Let's compare
streaming_out = torch.stack(output_frames).permute(1, 0, 2)
offline_output = offline_output[:, : streaming_out.shape[1], :]
print(
    "Результаты совпадают:" ,torch.allclose(
        offline_output,
        streaming_out,
        atol=1e-3,
    )
)



---



# Домашнее задание № 13

Выполните все задания в этом ноутбуке.

+ Мягкий дедлайн: `18.05.25 23:59`
+ Жесткий дедлайн: `25.05.25 23:59` (половина баллов)


После жесткого дедлайна задание не принимается.